# GMAIL CONNECTION

[Tutorial](https://developers.google.com/gmail/api/quickstart/python)

## AUTH FLOW


In [ ]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when| the authorization flow completes for the first
# time.
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            "gmail_credentials.json", SCOPES
        )
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
        token.write(creds.to_json())

## Actions

### List emails

In [ ]:
try:
    service = build("gmail", "v1", credentials=creds)
    response = service.users().messages().list(userId="me", maxResults=500,includeSpamTrash="true").execute()
    print(response)
    next_page_token = response['nextPageToken']
    messages = response["messages"]

    if not messages:
        print("No messages found.")

    print(f"Hay {len(messages)} emails")


except HttpError as error:
    print(f"An error occurred: {error}")

### READ RAW

In [ ]:
import base64
emailId="18f95cfbb7617da7"

try:
    service = build("gmail", "v1", credentials=creds)
    data = service.users().messages().get(userId="me", id=emailId,format="raw").execute()
    body = data["raw"]
    body = base64.urlsafe_b64decode(body)
    print(body)
except HttpError as error:
    print(f"An error occurred: {error}")


### READ

In [ ]:
import base64
emailId="18f95cfbb7617da7"

try:
    service = build("gmail", "v1", credentials=creds)
    data = service.users().messages().get(userId="me", id=emailId).execute()
    headers = data["payload"]["headers"]
    subject = ""
    fromEmail = ""
    toEmail = ""
    date = ""
    for header in headers:
        if "Subject" in header["name"]:
            subject = header["value"]
        elif "Date" in header["name"]:
            date = header["value"]
        elif "From" in header["name"]:
            fromEmail = header["value"]   
        elif "To" in header["name"]:
            toEmail = header["value"]    
    
    parts = data["payload"]["parts"]
    print(parts)
    body = parts[0]["body"]["data"]
    body = base64.urlsafe_b64decode(body)
    print(f"To: {toEmail}")
    print(f"From: {fromEmail}")
    print(f"Date: {date}")
    print(subject)
    print(body)
except HttpError as error:
    print(f"An error occurred: {error}")
